In [2]:
import rpy2.robjects as robjects
from rpy2.robjects.packages import importr
from rpy2.robjects import r

In [3]:
stm = importr('stm')
r('library(\'stm\')')
r('poliblogs<-read.csv("/Users/ninawang/Thesis/remote/THESIS2019/PART1/TopicModel/STM/stm_data.csv",header=TRUE)')

# preprocess documents
r("processed <- textProcessor(poliblogs$documents, metadata = poliblogs)")
r("out <- prepDocuments(processed$documents, processed$vocab, processed$meta)")
r("docs <- out$documents")
r("vocab <- out$vocab")
r("meta <-out$meta")



Building corpus... 
Converting to Lower Case... 
Removing punctuation... 
Removing stopwords... 
Removing numbers... 
Stemming... 
Creating Output... 
Removing 4994 of 28338 terms (4994 of 1514008 tokens) due to frequency 
Your corpus now has 6381 documents, 23344 terms and 1509014 tokens.

documents,docname,rating,date
...,...,...,...


In [4]:
r("First_STM <- stm(documents = out$documents, vocab = out$vocab," \
				"K = 30, prevalence =~ rating," \
				"max.em.its = 75, data = out$meta," \
				"init.type = \"Spectral\", verbose = FALSE)")

R object with classes: ('STM',) mapped to:
[ListSexpV..., FloatSexp..., ListSexpV..., ListSexpV..., ..., FloatSexp..., FloatSexp..., FloatSexp..., StrSexpVe...]
R object with classes: ('STM',) mapped to:
[ListSexpV..., FloatSexp..., ListSexpV..., ListSexpV..., ..., FloatSexp..., FloatSexp..., FloatSexp..., StrSexpVe...]
  sigma: <class 'rpy2.robjects.vectors.FloatMatrix'>
  R object with classes: ('matrix',) mapped to:
[10.775838, 5.186633, 1.900067, 6.219184, ..., 2.975426, 3.440233, 2.262695, 8.092087]
R object with classes: ('STM',) mapped to:
[ListSexpV..., FloatSexp..., ListSexpV..., ListSexpV..., ..., FloatSexp..., FloatSexp..., FloatSexp..., StrSexpVe...]
R object with classes: ('STM',) mapped to:
[ListSexpV..., FloatSexp..., ListSexpV..., ListSexpV..., ..., FloatSexp..., FloatSexp..., FloatSexp..., StrSexpVe...]
...
  convergence: <class 'rpy2.robjects.vectors.FloatMatrix'>
  R object with classes: ('matrix',) mapped to:
[-2.244553, -2.849441, 0.634662, -1.556587, ..., -0.911420, -0.997545, -0.147883, 0.103489]
  theta: <class 'rpy2.robjects.vectors.FloatMatrix'>
  R object with classes: ('matrix',) mapped to:
[0.283754, -0.041867, 0.065121, -0.025071, ..., -0.056130, -0.010495, 0.002267, 0.347154]
  eta: <class 'rpy2.robjects.vectors.FloatVector'>
  R object with classes: ('numeric',) mapped to:
[1234.886000]
  invsigma: <class 'rpy2.robjects.vectors.StrVector'>
  R object with classes: ('character',) mapped to:
['1.3.3']

In [5]:
# analyze results of covariates
r("predict_topics<-estimateEffect(formula = ~ rating, stmobj = First_STM," \
    "metadata = out$meta, uncertainty = \"Global\")")


R object with classes: ('estimateEffect',) mapped to:
[Lis..., Int..., Lan..., Str..., Lan..., Lis..., Lis..., Str...]
  parameters: <class 'rpy2.rinterface.ListSexpVector'>
  <rpy2.rinterface.ListSexpVector object at 0x11431f148> [RTYPES.VECSXP]
  topics: <class 'rpy2.rinterface.IntSexpVector'>
  <rpy2.rinterface.IntSexpVector object at 0x11431f8c8> [RTYPES.INTSXP]
  call: <class 'rpy2.rinterface.LangSexpVector'>
  <rpy2.rinterface.LangSexpVector object at 0x11431f488> [RTYPES.LANGSXP]
  uncertainty: <class 'rpy2.rinterface_lib.sexp.StrSexpVector'>
  <rpy2.rinterface_lib.sexp.StrSexpVector object at 0x11431f588> [RTYPES.STRSXP]
  formula: <class 'rpy2.rinterface.LangSexpVector'>
  <rpy2.rinterface.LangSexpVector object at 0x11431fb48> [RTYPES.LANGSXP]
  data: <class 'rpy2.rinterface.ListSexpVector'>
  <rpy2.rinterface.ListSexpVector object at 0x11431f608> [RTYPES.VECSXP]
  modelframe: <class 'rpy2.rinterface.ListSexpVector'>
  <rpy2.rinterface.ListSexpVector object at 0x11431f588> [RTYPES.VECSXP]
  varlist: <class 'rpy2.rinterface_lib.sexp.StrSexpVector'>
  <rpy2.rinterface_lib.sexp.StrSexpVector object at 0x11431f608> [RTYPES.STRSXP]

In [39]:
print("saving files")

folder = "data1"

r("dir.create(\""+folder+"\",showWarnings=FALSE,mode=\"0777\")")

# saving stm plot
r("pdf(file=\""+folder+"/"+"stm_plot.pdf\")")
r("plt = plot(First_STM)")
r("dev.off()")
# saving stm summary
r("capture.output(summary(First_STM), file=\""+folder+"/"+"stm_summary.txt\", append=FALSE)")
# saving covariate plot 
r("pdf(\""+folder+"/"+"cov_plot.pdf\")")
r("plot(predict_topics, covariate = \"rating\", topics = 1:30," \
        "model = First_STM, method = \"difference\"," \
        "cov.value1 = \"left\", cov.value2 = \"right\"," \
        "xlab = \"More Conservative ... More Liberal\"," \
        "main = \"Effect of Liberal vs. Conservative\"," \
        "xlim = c(-.03, .03), labeltype = \"custom\"," \
        "custom.labels = paste(\"Topic\",1:30))")
r("dev.off()")
# saving covariate summary
r("capture.output(summary(predict_topics), file=\""+folder+"/"+"cov_summary.txt\", append=FALSE)")


saving files


<rpy2.rinterface.NULLType object at 0x109e7c348> [RTYPES.NILSXP]

In [41]:
r("summ = summary(predict_topics)")
r("est <- predict_topics")

r("coef <- se <- pval <- rep(NA, 30)")
r("for (i in 1:30){" \
    "coef[i] <- est$parameters[[i]][[1]]$est[2]" \
    "\nse[i] <- est$parameters[[i]][[1]]$vcov[2,2]" \
    "\npval[i] <- summ$tables[[i]][8]}")

r("df <- data.frame(topic = 1:30, coef=coef, se=se, pval=pval)")
r("df <- df[order(df$coef),]")
r("df[order(df$coef),]")

topic,coef,se,pval
...,...,...,...


In [47]:
import rpy2.robjects as robjects

var = robjects.r['df']
print(var)

   topic          coef           se       pval
2      2 -0.0081510234 7.774489e-06 0.03975553
16    16 -0.0052795561 1.010841e-05 0.12363741
12    12 -0.0052180062 9.900130e-06 0.75396831
22    22 -0.0050874753 7.613071e-06 0.50790712
5      5 -0.0047868684 7.887345e-06 0.15286086
6      6 -0.0046145096 6.788809e-06 0.16086101
7      7 -0.0042195146 7.640455e-06 0.41402088
4      4 -0.0036510124 6.213881e-06 0.50459872
28    28 -0.0032847772 7.334086e-06 0.20651253
18    18 -0.0018657711 5.228382e-06 0.82210481
15    15 -0.0013368458 1.122325e-05 0.76173050
19    19 -0.0009821802 8.322477e-06 0.85025511
10    10 -0.0009071517 1.026595e-05 0.89211797
8      8 -0.0006393910 7.128236e-06 0.41310825
21    21 -0.0002630013 9.035893e-06 0.89189884
9      9  0.0004116006 7.219911e-06 0.98365771
27    27  0.0004864716 5.569686e-06 0.37167788
23    23  0.0007873332 6.871188e-06 0.97620868
11    11  0.0008253603 9.243572e-06 0.70945280
14    14  0.0010414965 6.580900e-06 0.61095815
29    29  0.0